# "[Drafted] -Optimizing pandas - Reducing 90% memory footprint - updated version"
> This post is base on https://www.dataquest.io/blog/pandas-big-data/ and updated with a new automated functions from https://github.com/ianozsvald/dtype_diet/blob/master/dtype_diet.py


- toc: true 
- badges: true
- comments: true
- categories: [python, pandas, optimization]
- hide: false

# Todo
- [ ] TWO options to automatically optimize pandas

In [1]:
#hide
%load_ext autoreload
%autoreload 2

import dtype_diet
from dtype_diet import *

In [3]:
#hide
import pandas as pd

# You need to download the dataset from https://data.world/dataquest/mlb-game-logs
df_gamelogs = pd.read_csv('../data/dataquest-mlb-game-logs.zip')
df_gamelogs.head()

c:\users\lrcno\anaconda3\envs\ge\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (12,13,14,15,19,20,81,83,85,87,93,94,95,96,97,98,99,100,105,106,108,109,111,112,114,115,117,118,120,121,123,124,126,127,129,130,132,133,135,136,138,139,141,142,144,145,147,148,150,151,153,154,156,157,160) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,na,1,FW1,na,1,0,...,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,na,1,WS3,na,1,20,...,Asa Brainard,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0,HTBF,Y
2,18710506,0,Sat,CL1,na,2,RC1,na,1,12,...,Pony Sager,6.0,birdg101,George Bird,7.0,stirg101,Gat Stires,9.0,NaN,Y
3,18710508,0,Mon,CL1,na,3,CH1,na,1,12,...,Ed Duffy,6.0,pinke101,Ed Pinkham,5.0,zettg101,George Zettlein,1.0,NaN,Y
4,18710509,0,Tue,BS1,na,2,TRO,na,1,9,...,Steve Bellan,5.0,pikel101,Lip Pike,3.0,cravb101,Bill Craver,6.0,HTBF,Y


We can check some basic info about the data with pandas `.info()` function

In [4]:
df_gamelogs.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: float64(77), int64(6), object(78)
memory usage: 859.4 MB


We can see the data has 171907 rows and 161 columns and 859.4 MB memory. Let's see how much we can optimize `dtype_diet`.

In [16]:
proposed_df = report_on_dataframe(df_gamelogs, unit="MB", unsafe=True)
proposed_df

,Current dtype,Proposed dtype,Current Memory (MB),Proposed Memory (MB),Ram Usage Improvement (MB),Ram Usage Improvement (%)
Column,,,,,,
date,int64,None,1343.148438,NaN,NaN,NaN
number_of_game,int64,Int8,1343.148438,335.880859,1007.267578,0.749930
day_of_week,object,category,10072.800781,168.725586,9904.075195,0.983249
v_name,object,category,10072.800781,349.552734,9723.248047,0.965297
v_league,object,category,9904.922852,168.718750,9736.204102,0.982966
...,...,...,...,...,...,...
h_player_9_id,object,category,11556.212891,400.092773,11156.120117,0.965379
h_player_9_name,object,category,11556.212891,400.092773,11156.120117,0.965379
h_player_9_def_pos,float64,category,11556.212891,400.092773,11156.120117,0.965379


In [19]:
new_df = optimize_dtypes(df_gamelogs, proposed_df, unsafe=True) # Avoid Type conversion error from int64 to int 8 with NA

In [20]:
print(f'Original df memory: {df_gamelogs.memory_usage(deep=True).sum()/1024/1024} MB')
print(f'Propsed df memory: {new_df.memory_usage(deep=True).sum()/1024/1024} MB')

Original df memory: 859.4271268844604 MB
Propsed df memory: 96.45247650146484 MB


In [ ]:
## hide
## collapse-hide
